In [1]:
import pandas as pd
import numpy as np
import os
import torch

data_dir = '../analysis_files/files/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

d:\Code\대학원\Paper\DAIC\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Train_data = pd.read_csv(os.path.join(data_dir, 'emotion_label.csv'))

In [3]:
Train_data.head()

,사람문장,감정,원인
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,분노,"진로,취업,직장"
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,분노,"진로,취업,직장"
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노,"진로,취업,직장"
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노,"진로,취업,직장"
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,분노,"진로,취업,직장"


In [4]:
column_values = {}
unique_values = Train_data['감정'].unique()
column_values['emotion'] = list(unique_values)
column_values['emotion']

['분노', '기쁨', '불안', '당황', '슬픔', '상처', '중립', '불만', '혐오', '좌절', '짜증', '무기력', nan]

In [5]:
# 감정 라벨링 리스트 생성
emotion_labels = ['분노', '기쁨', '불안', '당황', '슬픔', '상처', '중립', '불만', '혐오', '좌절', '짜증', '무기력', 0]

# 감정 라벨링 함수 정의
def label_emotion(emotion):
    if pd.isnull(emotion):  # NaN인 경우 0으로 라벨링
        return 0
    else:
        return emotion_labels.index(emotion) + 1

# 감정 column 라벨링 적용
Train_data['감정'] = Train_data['감정'].apply(label_emotion)

In [6]:
column_values = {}
unique_values = Train_data['원인'].unique()
column_values['cause'] = list(unique_values)
column_values['cause']

['진로,취업,직장',
 '대인관계',
 '연애,결혼,출산',
 '재정,은퇴,노후준비',
 '재정',
 '건강,죽음',
 '학업 및 진로',
 '학교폭력/따돌림',
 '직장, 업무 스트레스',
 '대인관계(부부, 자녀)',
 '가족관계',
 '건강',
 nan,
 '가족',
 '건강문제',
 '결혼',
 '경제적문제',
 '공부',
 '군대',
 '귀국',
 '남자친구',
 '남편',
 '대학',
 '문제',
 '부모',
 '사고',
 '사업',
 '생활',
 '성격',
 '시댁',
 '아르바이트',
 '애완동물',
 '어린시절',
 '여자친구',
 '연애',
 '유학',
 '육아',
 '음주',
 '이사',
 '이혼',
 '임신',
 '자각',
 '자녀',
 '전연인',
 '종교',
 '직장',
 '진로',
 '취업',
 '친구',
 '타인',
 '학교',
 '학업']

In [7]:
# '대인관계(부부, 자녀)' 값을 '가족'으로 변경
Train_data.loc[Train_data['원인'] == '대인관계(부부, 자녀)', '원인'] = '가족'

# '원인' 컬럼을 , 기준으로 원인1, 원인2, 원인3 컬럼으로 나누기
Train_data[['원인1', '원인2', '원인3']] = Train_data['원인'].str.split(',', expand=True)

# 공백 제거
Train_data['원인1'] = Train_data['원인1'].str.strip()
Train_data['원인2'] = Train_data['원인2'].str.strip()
Train_data['원인3'] = Train_data['원인3'].str.strip()

# 빈 값을 NaN으로 변경
Train_data[['원인1', '원인2', '원인3']] = Train_data[['원인1', '원인2', '원인3']].replace('', np.nan)
Train_data = Train_data.drop('원인', axis=1)

In [8]:
cause_mapping = {
    '가족': ['대인관계(부부, 자녀)','가족관계','가족','남편','부모','시댁','애완동물','자녀'],
    '건강': ['건강', '죽음', '건강문제','음주', '사고'],
    '결혼': ['결혼', '이혼'],
    '대인관계': ['어린시절', '대인관계', '성격', '친구','타인'],
    '사회': ['군대', '생활', '종교'],
    '연애': ['연애','남자친구','여자친구','전애인'],
    '재정': ['재정', '노후준비', '경제적문제'],
    '직장': ['직장','업무 스트레스','사업','아르바이트','은퇴', '취업'],
    '진로': ['진로', '귀국','대학','유학','자존감저하','절망감','좌절','죄책감','창피함','후회','비관적'],
    '출산': ['출산', '육아', '임신'],
    '학교': ['학교','학업 및 진로', '학교폭력/따돌림','공부','학업'],
}

In [9]:

# 1. Train_data의 원인1, 원인2, 원인3 각각에 값이 있는지 확인
for column in ['원인1', '원인2', '원인3']:
    Train_data[column] = Train_data[column].fillna('')

# 2. 빈 칸인 컬럼은 pass, 값이 있는 컬럼만 다음 단계로 넘어가기
for index, row in Train_data.iterrows():
    causes = [row['원인1'], row['원인2'], row['원인3']]
    if all(not cause for cause in causes):
        continue

    # 3. 값이 있다면 cause_mapping에 따라 컬럼들을 매칭해주고, 빈 칸이 아닌 값 중에 cause_mapping에 매칭되는 컬럼이 없다면 '기타'로 표시
    new_causes = []
    for cause in causes:
        if cause:
            matched_cause = False
            for key, values in cause_mapping.items():
                if cause in values:
                    new_causes.append(key)
                    matched_cause = True
                    break
            if not matched_cause:
                new_causes.append('기타')
        else:
            continue
            
    # 4. 기타는 0, 가족~학교까지는 1~11까지 컬럼들로 매칭
    label_mapping = {
        '기타': 0,
        '가족': 1,
        '건강': 2,
        '결혼': 3,
        '대인관계': 4,
        '사회': 5,
        '연애': 6,
        '재정': 7,
        '직장': 8,
        '진로': 9,
        '출산': 10,
        '학교': 11
    }
    for i, cause in enumerate(new_causes):
        Train_data.loc[index, f'원인{i+1}'] = label_mapping[cause]

In [10]:
Train_data.head()

,사람문장,감정,원인1,원인2,원인3
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,1,9,8,8
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,1,9,8,8
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,1,9,8,8
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,1,9,8,8
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,1,9,8,8


In [15]:
Train_data = Train_data.rename(columns={'사람문장': 'sentence', '감정': 'emotion'})
Train_data['cause'] = Train_data[['원인1', '원인2', '원인3']].astype(str).apply(lambda x: ', '.join(filter(None, x)), axis=1)
Train_data['cause'] = Train_data['cause'].replace('', np.nan).fillna(Train_data['cause'])

In [16]:
Train_data = Train_data.drop('원인1', axis=1)
Train_data = Train_data.drop('원인2', axis=1)
Train_data = Train_data.drop('원인3', axis=1)

In [17]:
Train_data.head()

,sentence,emotion,cause
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,1,"9, 8, 8"
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,1,"9, 8, 8"
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,1,"9, 8, 8"
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,1,"9, 8, 8"
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,1,"9, 8, 8"


In [ ]:
Train_data.to_csv(os.path.join(data_dir, 'train_data(emotion, cause).csv'), encoding = 'cp949', index=False)